Cargo los datos sobre temperaturas durante el año 2023 de Barcelona desde la estación de Drassanes que tiene un criterio de temperatura a nivel ciudad.

In [18]:
import pandas as pd

# Cargo los archivos CSV
df1 = pd.read_csv('temp_012023_062023.csv')
df2 = pd.read_csv('temp_072023_122023.csv')

# Concateno los DataFrames por filas
df_temp_2023 = pd.concat([df1, df2], axis=0, ignore_index=True)

df_temp_2023.head()

,fecha,indicativo,nombre,provincia,altitud,tmed,prec,tmin,horatmin,tmax,...,dir,velmedia,racha,horaracha,sol,presMax,horaPresMax,presMin,horaPresMin,hrMedia
0,2023-01-01,0200E,"BARCELONA, FABRA",BARCELONA,408,"13,0","0,0","10,8",08:47,"15,3",...,27,"2,2","6,9",00:05,"2,2","977,6",11,"975,7",Varias,68.0
1,2023-01-02,0200E,"BARCELONA, FABRA",BARCELONA,408,"12,8","0,1","10,9",22:38,"14,7",...,5,"2,5","7,2",22:01,"1,4","976,7",24,"974,0",08,74.0
2,2023-01-03,0200E,"BARCELONA, FABRA",BARCELONA,408,"12,5","0,0","9,0",Varias,"16,0",...,33,"2,2","10,3",21:13,"4,6","981,7",Varias,"976,7",00,72.0
3,2023-01-04,0200E,"BARCELONA, FABRA",BARCELONA,408,"13,4","0,0","9,1",06:06,"17,7",...,34,"3,9","7,8",00:10,"9,1","983,7",10,"981,6",Varias,63.0
4,2023-01-05,0200E,"BARCELONA, FABRA",BARCELONA,408,"12,7","0,0","8,9",23:44,"16,5",...,30,"4,2","9,2",23:33,"9,1","982,2",00,"975,7",24,53.0


El dataframe está compuesto por 365 filas y 20 columnas

In [19]:
df_temp_2023.shape 

(365, 21)

Hay columnas que no contemplo utilizar por lo que decido eliminarlas para aligerar el dataframe.

In [20]:
df_temp_2023= df_temp_2023.drop(columns=['indicativo', 'nombre', 'altitud', 'dir', 'racha', 'horaracha', 'horatmin', 'horatmax', 'sol','presMax', 'horaPresMax', 'presMin', 'horaPresMin' ])

df_temp_2023.head()

,fecha,provincia,tmed,prec,tmin,tmax,velmedia,hrMedia
0,2023-01-01,BARCELONA,"13,0","0,0","10,8","15,3","2,2",68.0
1,2023-01-02,BARCELONA,"12,8","0,1","10,9","14,7","2,5",74.0
2,2023-01-03,BARCELONA,"12,5","0,0","9,0","16,0","2,2",72.0
3,2023-01-04,BARCELONA,"13,4","0,0","9,1","17,7","3,9",63.0
4,2023-01-05,BARCELONA,"12,7","0,0","8,9","16,5","4,2",53.0


Reviso las características de cada columna, observo varios detalles sobre ellas que considero tratar:

* `fecha` es del tipo `object` y el formato debería ser `datetime`
* Hay varias columas que aparecen como tipo `object` al utiizar una coma como decimal y las transformaré como float con punto.


In [21]:
df_temp_2023.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 365 entries, 0 to 364
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   fecha      365 non-null    object 
 1   provincia  365 non-null    object 
 2   tmed       365 non-null    object 
 3   prec       365 non-null    object 
 4   tmin       365 non-null    object 
 5   tmax       365 non-null    object 
 6   velmedia   365 non-null    object 
 7   hrMedia    361 non-null    float64
dtypes: float64(1), object(7)
memory usage: 22.9+ KB


Realizo la transformación de la columna `fecha` al tipo `datetime`

In [22]:
df_temp_2023['fecha']= pd.to_datetime(df_temp_2023['fecha'], format='%Y-%m-%d')
print(df_temp_2023.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                 object
prec                 object
tmin                 object
tmax                 object
velmedia             object
hrMedia             float64
dtype: object


He encontrado un error al intentar cambiar las columnas a `float`, en el dataframe existe una columna con el valor `Ip` que me impide realizarlo, realizo la siguiente tarea para encontrarla.

In [23]:
columnas_con_ip = df_temp_2023.columns[df_temp_2023.apply(lambda col: "Ip" in col.values)].tolist()

print(columnas_con_ip)

['prec']


Reviso la cantidad de registros con ese valor

In [24]:
df_temp_2023['prec'].value_counts()

prec
0,0     266
Ip       36
0,2      11
0,6       5
0,1       5
0,8       3
0,5       2
1,3       2
1,0       2
1,1       2
2,0       1
33,7      1
3,8       1
2,9       1
32,4      1
19,0      1
22,0      1
4,7       1
17,8      1
16,9      1
8,3       1
2,6       1
0,4       1
3,2       1
3,1       1
9,1       1
9,7       1
7,4       1
3,5       1
3,9       1
1,9       1
1,6       1
28,1      1
7,5       1
0,7       1
0,3       1
19,7      1
4,3       1
15,6      1
6,3       1
1,2       1
Name: count, dtype: int64

In [25]:
import numpy as np

# Ordeno el dataframe por fecha
df_temp_2023 = df_temp_2023.sort_values('fecha')

# Primero de convierten los valores "Ip" a NaN
df_temp_2023['prec'] = df_temp_2023['prec'].replace('Ip', np.nan)

# Se imputan los valores NaN con el valor numérico anterior usando ffill()
df_temp_2023['prec'] = df_temp_2023['prec'].ffill()

Transformo las columnas 'tmed', 'prec', 'tmin', 'tmax' al tipo float, modificando el símbolo de coma por punto.

In [26]:
columnas_a_float = ['tmed', 'prec', 'tmin', 'tmax', 'velmedia']

# Reemplazo las comas por puntos para que pueda realizarse el cambio al tipo float
for col in columnas_a_float:
    df_temp_2023[col] = df_temp_2023[col].str.replace(',', '.')  

df_temp_2023[columnas_a_float] = df_temp_2023[columnas_a_float].astype(float)

print(df_temp_2023.dtypes)

fecha        datetime64[ns]
provincia            object
tmed                float64
prec                float64
tmin                float64
tmax                float64
velmedia            float64
hrMedia             float64
dtype: object


In [27]:
df_temp_2023.isnull().sum()

fecha        0
provincia    0
tmed         0
prec         0
tmin         0
tmax         0
velmedia     0
hrMedia      4
dtype: int64

Analizo los valores nulos de la variable `tmed`; observo que aparecen varias columnas en ese registros con valores faltantes.

In [28]:
nulos_hrmed = df_temp_2023['hrMedia'].isnull()
df_temp_2023[nulos_hrmed]

,fecha,provincia,tmed,prec,tmin,tmax,velmedia,hrMedia
12,2023-01-13,BARCELONA,11.6,0.0,7.8,15.4,3.9,NaN
53,2023-02-23,BARCELONA,10.0,4.3,8.5,11.5,1.7,NaN
75,2023-03-17,BARCELONA,12.2,0.0,9.8,14.5,3.6,NaN
232,2023-08-21,BARCELONA,32.2,0.0,27.6,36.8,2.2,NaN


La idea de la imputación en este caso es mantener la continuidad de la serie. Imputo los valores faltantes utilizando una función, que en caso de que encuentre un valor nulo en una columna, buscará el valor correspondiente en la fila anterior (día anterior). Si el valor anterior existe, se utiliza para reemplazar el valor nulo. 

In [29]:
# Ordeno el DataFrame por fecha para asegurar la correcta imputación
df_temp_2023 = df_temp_2023.sort_values('fecha')

# Selecciono las columnas a imputar 
columnas_a_imputar = ['hrMedia']

# Se itera sobre las filas y las columnas a imputar
for index, row in df_temp_2023.iterrows():
    for columna in columnas_a_imputar:
        if pd.isna(row[columna]):
            # Buscar el valor del día anterior en la misma columna
            valor_anterior = df_temp_2023.loc[index - 1, columna]
            # Imputar si el valor anterior existe
            if not pd.isna(valor_anterior):
                df_temp_2023.loc[index, columna] = valor_anterior

Compruebo que los valores ha sido imputados totalmente.

In [30]:
df_temp_2023.isnull().sum()

fecha        0
provincia    0
tmed         0
prec         0
tmin         0
tmax         0
velmedia     0
hrMedia      0
dtype: int64

- Renombro las columna `provincia` por `ciudad`.

In [31]:
df_temp_2023= df_temp_2023.rename(columns={'provincia': 'ciudad', 'hrMedia': 'hrmedia'})
df_temp_2023= df_temp_2023[['fecha', 'ciudad', 'tmin', 'tmax', 'tmed', 'prec','hrmedia', 'velmedia']]
df_temp_2023.head()


,fecha,ciudad,tmin,tmax,tmed,prec,hrmedia,velmedia
0,2023-01-01,BARCELONA,10.8,15.3,13.0,0.0,68.0,2.2
1,2023-01-02,BARCELONA,10.9,14.7,12.8,0.1,74.0,2.5
2,2023-01-03,BARCELONA,9.0,16.0,12.5,0.0,72.0,2.2
3,2023-01-04,BARCELONA,9.1,17.7,13.4,0.0,63.0,3.9
4,2023-01-05,BARCELONA,8.9,16.5,12.7,0.0,53.0,4.2


Codifico la columna `ciudad` utilizando el valor `1` para poder relacionarlo en la base de datos con la tabla que corresponda a las ciudades

In [32]:
mapeo_ciudad= {"BARCELONA": "1"}

df_temp_2023['ciudad']= df_temp_2023['ciudad'].map(mapeo_ciudad)
df_temp_2023.head()

,fecha,ciudad,tmin,tmax,tmed,prec,hrmedia,velmedia
0,2023-01-01,1,10.8,15.3,13.0,0.0,68.0,2.2
1,2023-01-02,1,10.9,14.7,12.8,0.1,74.0,2.5
2,2023-01-03,1,9.0,16.0,12.5,0.0,72.0,2.2
3,2023-01-04,1,9.1,17.7,13.4,0.0,63.0,3.9
4,2023-01-05,1,8.9,16.5,12.7,0.0,53.0,4.2


Exporto el dataframe a un archivo en formato `csv`

In [33]:
df_temp_2023.to_csv('temp_bcn_2023.csv', index= False, encoding='utf-8')

Resumen del dataframe creado:

- Se eliminaron las columnas 'indicativo', 'nombre', 'hrMax', 'horaHrMax', 'hrMin', 'horaHrMin', 'altitud', 'dir', 'racha', 'horaracha', 'horatmin', 'horatmax'.
- Se identificó la columna con el valor `Ip` que impedía cambiar el tipo `float` en las columnas que lo necesitan
- Se realizaron transformaciones de tipo en las columnas 'tmed', 'prec', 'tmin', 'tmax', 'velmedia'.
- Se transformó la columna 'fecha' a tipo datetime
- Se trataron los valores nulos mediante la imputación del registro del día anterior al faltante, buscando mantener la continuidad de la serie. No hay valores faltantes.
- Se renombró la columna 'provincia' por 'ciudad' y se asignó el valor 1 para que sea utilizado en un modelo de tablas relaciones como identificador(id).
- Se exporto el dataframe a un archivo en formato csv.